In [ ]:
#!pip install pandas geopy ortools numpy permutations combinations

In [ ]:
# Import all libraries
import pandas as pd
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import numpy as np
from itertools import permutations, combinations

In [ ]:
# Read in drivers/passengers csv's
drivers = pd.read_csv("drivers.csv")
roster_passengers = pd.read_csv("passengers.csv")

In [ ]:
# Input the drivers coming
driver_names = input("Input Names (comma separated): ").split(",")  # Split input by commas to get a list
driver_names = [name.strip() for name in driver_names]  # Strip any extra whitespace from each name
practice_drivers = drivers[drivers["Name"].isin(driver_names)]

In [ ]:
# Input the passengers coming
passenger_names = input("Input Names (comma separated): ").split(",")  # Split input by commas to get a list
passenger_names = [name.strip() for name in passenger_names]  # Strip any extra whitespace from each name
practice_passengers = roster_passengers[roster_passengers["Name"].isin(passenger_names)]

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [ ]:
# Create a fcn that gets the latitude and longitude for a  given address
def get_lat_lon(address):
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

In [ ]:
# Get coordinates for drumlins tennis club (where practices are)
# Define the end location
drumlins = "800 Nottingham Rd, Syracuse, NY"
# Get latitude and longitude of the end location
end_coords = get_lat_lon(drumlins)

In [ ]:
# Get the latitude and longitude for all driver start locations
driver_locs = practice_drivers['Address'].apply(lambda x: pd.Series(get_lat_lon(x)))
practice_drivers.loc[:, "lat"] = driver_locs[0]
practice_drivers.loc[:, "lon"] = driver_locs[1]

In [ ]:
# Get the latitude and longitude for all passenger locations
passenger_locs = practice_passengers['Address'].apply(lambda x: pd.Series(get_lat_lon(x)))
practice_passengers.loc[:, "lat"] = passenger_locs[0]
practice_passengers.loc[:, "lon"] = passenger_locs[1]

In [ ]:
# Function to calculate the total distance for a driver's route
def calculate_route_distance(driver_loc, passenger_locs):
    total_distance = 0 # Initialize distance
    route = [driver_loc] + passenger_locs + [end_coords]  # Start at driver, pickup passenger(s), end at destination
    for i in range(len(route) - 1): # For each instance in the second-to-last stop until the nth stpo
        total_distance += geodesic(route[i], route[i+1]).miles # Calculate the total distance between stops
        # Use += to calculate cumulative distance
    return total_distance

In [ ]:
# Helper function to get (lat, lon) tuple from DataFrame row
def get_lat_lon(row):
    return (row['lat'], row['lon'])

In [ ]:
# Function to generate all valid passenger assignments given driver capacities
def generate_valid_assignments(drivers, passengers):
    passenger_count = len(passengers)
    driver_capacities = drivers['Passengers'].tolist()

    # Create all possible ways to split passengers between drivers
    valid_assignments = []

    def recursive_split(remaining_passengers, current_assignment, current_driver):
        if current_driver == len(driver_capacities): # Has the current driver reached capacity for all drivers
            if len(remaining_passengers) == 0: # If there are no remaining passengers, set that as the assignment
                valid_assignments.append(current_assignment)
            return

        max_capacity = driver_capacities[current_driver] # Set the maximum number of passengers a driver can take
        for subset_size in range(1, min(len(remaining_passengers), max_capacity) + 1):
          # For each possible combination of passengers in the range of 1 to the remaining numer of passengers or maximum capacity
            for passenger_subset in combinations(remaining_passengers, subset_size):
              # For each possible passenger combinations of the all passengers create unique pairings for each driver
                remaining = [p for p in remaining_passengers if p not in passenger_subset]
                # Find the remaining passengers if they are not placed in a car
                recursive_split(remaining, current_assignment + [list(passenger_subset)], current_driver + 1)
                # Apply the function wiht the remaining people, the current passengers assignment, and the next driver

    all_passengers = list(range(passenger_count))  # Use passenger indices to assign them
    recursive_split(all_passengers, [], 0)

    return valid_assignments

In [ ]:
# Function to evaluate all possible routes for all drivers
def find_best_routes(drivers, passengers):
    # Initialize the best assignment/minimal total distance
    best_assignment = None
    best_total_distance = float('inf')

    # Apply generate_valid_assignments fcn for the drivers/passengers coming to practice
    assignments = generate_valid_assignments(drivers, passengers)

    for assignment in assignments: # Loop through each possible assignment
        total_distance = 0 # Initialize a total_distance
        valid = True # Check if all assignments are valid

        # Calculate route distance for each driver
        for driver_idx, passenger_indices in enumerate(assignment):
          # Apply get_lat_lon fcn to get driver/passenger latitudes and longitudes
            driver_loc = get_lat_lon(drivers.iloc[driver_idx])
            passenger_locs = [get_lat_lon(passengers.iloc[i]) for i in passenger_indices]

            # For each permutation of the passenger locations, calculate route distance
            best_route_distance = float('inf')
            for perm in permutations(passenger_locs): # For each possible permutation
                route_distance = calculate_route_distance(driver_loc, list(perm)) # Calculate the total_distance for each possible permutation
                best_route_distance = min(best_route_distance, route_distance) # Find the lowest total distance

            total_distance += best_route_distance

            # If at any point total distance exceeds the current best, skip this assignment (because it will be longer)
            if total_distance > best_total_distance:
                valid = False
                break

        # If this assignment is the best, keep it
        if valid and total_distance < best_total_distance:
            best_total_distance = total_distance
            best_assignment = assignment

    return best_assignment, best_total_distance

In [ ]:
# Apply fcn to find the optimal routes
best_assignment, best_total_distance = find_best_routes(practice_drivers, practice_passengers)

In [ ]:
# Print out results with pickup locations
for driver_idx, passenger_indices in enumerate(best_assignment):
    driver_name = practice_drivers.iloc[driver_idx]['Name'] # Get driver names
    assigned_passengers = practice_passengers.iloc[passenger_indices].copy() # Get passengers assigned to drivers
    # Remove ", Syracuse, NY"
    assigned_passengers.loc[:, 'Address'] = assigned_passengers['Address'].str.replace(", Syracuse, NY", "", regex=False).str.strip()
    # Get passenger addresses
    passengers = assigned_passengers[['Name', "Address"]] # Subset name/address
    print(f"{driver_name}:") # Print the driver's name and the passenger/address for each assigned passenger to that driver
    for index, row in assigned_passengers.iterrows():
      print(f"{row['Name']} ({row['Address']})")